In [1]:
pip install pygad

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 1.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import pygad
from google.colab import drive
from google.colab import files

In [3]:
drive.mount('/content/Drive/')

Mounted at /content/Drive/


In [4]:
data_18 = pd.read_csv('/content/Drive/MyDrive/논문/model_18.csv')
data_18.head()

,temp,wind,humidity,barometer,day_power,sky_cover
0,0.312052,0.234784,0.778769,0.638520,0.058824,2
1,0.369939,0.412924,0.725910,0.424439,0.058824,3
2,0.352761,0.572187,0.709644,0.296313,0.058824,5
3,0.348520,0.334605,0.751113,0.118489,0.029412,3
4,0.348466,0.313384,0.735478,0.160241,0.029412,6


In [5]:
len(data_18)

1332

* temp, wind, humidity 퍼지화

In [6]:
data = data_18['temp']
X_fuz = []
for i in range(len(data)-1):
  spread = np.abs(data[i] - data[i+1]) / 2
  X_fuz.append([data[i] - spread, data[i], data[i] + spread])
X_fuz.append([data[len(data)-1] - spread, data[len(data)-1], data[len(data)-1] + spread])

In [7]:
X_fuz = np.array(X_fuz)
X_fuz

array([[0.28310838, 0.31205181, 0.34099523],
       [0.36134969, 0.36993865, 0.37852761],
       [0.35064017, 0.35276074, 0.3548813 ],
       ...,
       [0.22226483, 0.25710634, 0.29194785],
       [0.30696575, 0.32678937, 0.34661299],
       [0.26731851, 0.28714213, 0.30696575]])

In [8]:
data = data_18['wind']
Z_fuz = []
for i in range(len(data)-1):
  spread = np.abs(data[i] - data[i+1]) / 2
  Z_fuz.append([data[i] - spread, data[i], data[i] + spread])
Z_fuz.append([data[len(data)-1] - spread, data[len(data)-1], data[len(data)-1] + spread])

In [9]:
Z_fuz = np.array(Z_fuz)
Z_fuz

array([[0.14571496, 0.23478449, 0.32385403],
       [0.33329186, 0.41292356, 0.49255527],
       [0.4533958 , 0.57218697, 0.69097814],
       ...,
       [0.00877193, 0.10331384, 0.19785575],
       [0.26250812, 0.29239766, 0.3222872 ],
       [0.20272904, 0.23261858, 0.26250812]])

In [10]:
data = data_18['humidity']
M_fuz = []
for i in range(len(data)-1):
  spread = np.abs(data[i] - data[i+1]) / 2
  M_fuz.append([data[i] - spread, data[i], data[i] + spread])
M_fuz.append([data[len(data)-1] - spread, data[len(data)-1], data[len(data)-1] + spread])

In [11]:
M_fuz = np.array(M_fuz)
M_fuz

array([[0.75233957, 0.77876915, 0.80519874],
       [0.71777698, 0.72590998, 0.73404298],
       [0.68890934, 0.70964398, 0.73037862],
       ...,
       [0.78451541, 0.80918683, 0.83385826],
       [0.72049462, 0.75984399, 0.79919335],
       [0.6417959 , 0.68114526, 0.72049462]])

* sky cover 퍼지화

In [12]:
W_fuz = np.array([data_18['sky_cover'] - 1, data_18['sky_cover'], data_18['sky_cover'] + 1]).T
W_fuz

array([[1, 2, 3],
       [2, 3, 4],
       [4, 5, 6],
       ...,
       [1, 2, 3],
       [6, 7, 8],
       [3, 4, 5]])

In [13]:
row, col = W_fuz.shape

In [14]:
for i in range(row):    # 1~8 이므로 1보다 작거나 8보다 큰 부분 수정
  if W_fuz[i][0] < 1:
    W_fuz[i][0] = 1
  elif W_fuz[i][2] >8:
    W_fuz[i][2] = 8

* day_power crisp

In [15]:
Y_fuz = np.array([data_18['day_power'], data_18['day_power'], data_18['day_power']]).T
Y_fuz

array([[0.05882353, 0.05882353, 0.05882353],
       [0.05882353, 0.05882353, 0.05882353],
       [0.05882353, 0.05882353, 0.05882353],
       ...,
       [0.08823529, 0.08823529, 0.08823529],
       [0.        , 0.        , 0.        ],
       [0.17647059, 0.17647059, 0.17647059]])

In [16]:
Y_fuz.shape

(1332, 3)

In [17]:
XW_fuz = X_fuz * W_fuz
XZ_fuz = X_fuz * Z_fuz
WZ_fuz = W_fuz * Z_fuz
XWZ_fuz = X_fuz * W_fuz * Z_fuz

In [18]:
one = np.ones((1332,3))

## GA 입력을 위해 삼각퍼지수 쪼개기

In [19]:
X_l = []
X_m = []
X_r = []

M_l = []
M_m = []
M_r = []

Y_l = []
Y_m = []
Y_r = []

W_l = []
W_m = []
W_r = []

Z_l = []
Z_m = []
Z_r = []

XW_l = []
XW_m = []
XW_r = []

XZ_l = []
XZ_m = []
XZ_r = []

WZ_l = []
WZ_m = []
WZ_r = []

XWZ_l = []
XWZ_m = []
XWZ_r = []

In [20]:
for i in range(len(data_18)):
  X_l.append(X_fuz[i][0])
  X_m.append(X_fuz[i][1])
  X_r.append(X_fuz[i][2])

In [21]:
for i in range(len(data_18)):
  M_l.append(M_fuz[i][0])
  M_m.append(M_fuz[i][1])
  M_r.append(M_fuz[i][2])

for i in range(len(data_18)):
  Y_l.append(Y_fuz[i][0])
  Y_m.append(Y_fuz[i][1])
  Y_r.append(Y_fuz[i][2])

for i in range(len(data_18)):
  W_l.append(W_fuz[i][0])
  W_m.append(W_fuz[i][1])
  W_r.append(W_fuz[i][2])

for i in range(len(data_18)):
  Z_l.append(Z_fuz[i][0])
  Z_m.append(Z_fuz[i][1])
  Z_r.append(Z_fuz[i][2])

for i in range(len(data_18)):
  XW_l.append(XW_fuz[i][0])
  XW_m.append(XW_fuz[i][1])
  XW_r.append(XW_fuz[i][2])

for i in range(len(data_18)):
  XZ_l.append(XZ_fuz[i][0])
  XZ_m.append(XZ_fuz[i][1])
  XZ_r.append(XZ_fuz[i][2])

for i in range(len(data_18)):
  WZ_l.append(WZ_fuz[i][0])
  WZ_m.append(WZ_fuz[i][1])
  WZ_r.append(WZ_fuz[i][2])

for i in range(len(data_18)):
  XWZ_l.append(XWZ_fuz[i][0])
  XWZ_m.append(XWZ_fuz[i][1])
  XWZ_r.append(XWZ_fuz[i][2])

----

# L2-metric

## M 추정

In [48]:
# GA parameter 세팅

sol_per_pop = 100
num_genes = 8               # bias 포함
init_range_low = -1
init_range_high = 1


num_generations = 100
num_parents_mating = 5
gene_type = float
parent_selection_type = "rws"  # rws, rank
keep_parents = 3

crossover_type = "single_point"
mutation_type = "random"
mutation_percent_genes = 5

In [49]:
one = np.ones((1332)) # bias

In [24]:
m_data_l = np.array([one, X_l, W_l, Z_l, XW_l, XZ_l, WZ_l, XWZ_l]).T
m_data_m = np.array([one, X_m, W_m, Z_m, XW_m, XZ_m, WZ_m, XWZ_m]).T
m_data_r = np.array([one, X_r, W_r, Z_r, XW_r, XZ_r, WZ_r, XWZ_r]).T

In [25]:
m_data_l.shape

(1332, 8)

In [50]:
def fitness_function_factory(X_l, X_m, X_r, Y_l, Y_m, Y_r):
      def fitness_function(ga_instance,solution, solution_idx):

          Y_hat_l = solution[0]*X_l[:,0] +solution[1]*X_l[:,1] +solution[2]*X_l[:,2] +solution[3]*X_l[:,3] +solution[4]*X_l[:,4] +solution[5]*X_l[:,5] +solution[6]*X_l[:,6] +solution[7]*X_l[:,7]
          Y_hat_m = solution[0]*X_m[:,0] +solution[1]*X_m[:,1] +solution[2]*X_m[:,2] +solution[3]*X_m[:,3] +solution[4]*X_m[:,4] +solution[5]*X_m[:,5] +solution[6]*X_m[:,6] +solution[7]*X_m[:,7]
          Y_hat_r = solution[0]*X_r[:,0] +solution[1]*X_r[:,1] +solution[2]*X_r[:,2] +solution[3]*X_r[:,3] +solution[4]*X_r[:,4] +solution[5]*X_r[:,5] +solution[6]*X_r[:,6] +solution[7]*X_r[:,7]
          error = ((Y_l - Y_hat_l) ** 2).sum() + ((Y_m - Y_hat_m) ** 2).sum() + ((Y_r - Y_hat_r) ** 2).sum()
          fitness = 1/(error)
          return fitness

      return fitness_function

In [51]:
lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(m_data_l, m_data_m, m_data_r, M_l, M_m, M_r),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_genes,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        gene_type=gene_type,
                        random_seed=18)
lr_ga.run()
sol=lr_ga.best_solution()[0]
sol

array([ 0.69018803, -0.28883019,  0.11026186, -0.87526932, -0.17750213,
        1.19110617, -0.04794752,  0.22333366])

In [52]:
print(sol)

[ 0.69018803 -0.28883019  0.11026186 -0.87526932 -0.17750213  1.19110617
 -0.04794752  0.22333366]


In [53]:
M_hat = 0.6902 + -0.2888*X_fuz + 0.1103*W_fuz + -0.8753*Z_fuz + -0.1775*XW_fuz + 1.1911*XZ_fuz + -0.0479*WZ_fuz + 0.2233*XWZ_fuz

----

## Y 추정

In [54]:
# GA parameter 세팅

sol_per_pop = 100
num_genes = 3              # bias 포함
init_range_low = 1
init_range_high = -1


num_generations = 100
num_parents_mating = 5
gene_type = float
parent_selection_type = "rws"  # rws, rank
keep_parents = 3

crossover_type = "single_point"
mutation_type = "random"
mutation_percent_genes = 5

In [55]:
one = np.ones((1332)) # bias

In [56]:
y_data_l = np.array([one, X_l, M_l]).T
y_data_m = np.array([one, X_m, M_m]).T
y_data_r = np.array([one, X_r, M_r]).T

In [57]:
y_data_l.shape

(1332, 3)

In [58]:
def fitness_function_factory(X_l, X_m, X_r, Y_l, Y_m, Y_r):
      def fitness_function(ga_instance,solution, solution_idx):

          Y_hat_l = solution[0]*X_l[:,0] +solution[1]*X_l[:,1] +solution[2]*X_l[:,2]
          Y_hat_m = solution[0]*X_m[:,0] +solution[1]*X_m[:,1] +solution[2]*X_m[:,2]
          Y_hat_r = solution[0]*X_r[:,0] +solution[1]*X_r[:,1] +solution[2]*X_r[:,2]
          error = ((Y_l - Y_hat_l) ** 2).sum() + ((Y_m - Y_hat_m) ** 2).sum() + ((Y_r - Y_hat_r) ** 2).sum()
          fitness = 1/(error)
          return fitness

      return fitness_function

In [59]:
lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(y_data_l,y_data_m, y_data_r, Y_l, Y_m, Y_r),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_genes,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        gene_type=gene_type,
                        random_seed=18)
lr_ga.run()
sol=lr_ga.best_solution()[0]
sol

array([ 0.53906806,  0.56210726, -0.77535926])

In [60]:
print(sol)

[ 0.53906806  0.56210726 -0.77535926]


## MSE & MAE

In [61]:
one_fuz = np.ones((1332,3))

In [62]:
Y_fuz_hat = 0.5391 *one_fuz + 0.5621*X_fuz + -0.7754*M_hat

* MSE

In [63]:
mse = ((Y_fuz - Y_fuz_hat) ** 2).sum(axis=1).sum() / 1332
print(mse)

0.107493083136409


* MAE

In [64]:
abs = np.abs(Y_fuz - Y_fuz_hat)
mae = abs.sum(axis=1).sum()/len(data_18)
print(mae)

0.45641610788447856


---

# L1-metric

## M 추정

In [123]:
# GA parameter 세팅

sol_per_pop = 200
num_genes = 8               # bias 포함
init_range_low = -1
init_range_high = 1


num_generations = 200
num_parents_mating = 5
gene_type = float
parent_selection_type = "rws"  # rws, rank
keep_parents = 3

crossover_type = "single_point"
mutation_type = "random"
mutation_percent_genes = 5

In [124]:
one = np.ones((1332)) # bias

In [125]:
m_data_l = np.array([one, X_l, W_l, Z_l, XW_l, XZ_l, WZ_l, XWZ_l]).T
m_data_m = np.array([one, X_m, W_m, Z_m, XW_m, XZ_m, WZ_m, XWZ_m]).T
m_data_r = np.array([one, X_r, W_r, Z_r, XW_r, XZ_r, WZ_r, XWZ_r]).T

In [126]:
m_data_l.shape

(1332, 8)

In [127]:
def fitness_function_factory(X_l, X_m, X_r, Y_l, Y_m, Y_r):
      def fitness_function(ga_instance,solution, solution_idx):

          Y_hat_l = solution[0]*X_l[:,0] +solution[1]*X_l[:,1] +solution[2]*X_l[:,2] +solution[3]*X_l[:,3] +solution[4]*X_l[:,4] +solution[5]*X_l[:,5] +solution[6]*X_l[:,6] +solution[7]*X_l[:,7]
          Y_hat_m = solution[0]*X_m[:,0] +solution[1]*X_m[:,1] +solution[2]*X_m[:,2] +solution[3]*X_m[:,3] +solution[4]*X_m[:,4] +solution[5]*X_m[:,5] +solution[6]*X_m[:,6] +solution[7]*X_m[:,7]
          Y_hat_r = solution[0]*X_r[:,0] +solution[1]*X_r[:,1] +solution[2]*X_r[:,2] +solution[3]*X_r[:,3] +solution[4]*X_r[:,4] +solution[5]*X_r[:,5] +solution[6]*X_r[:,6] +solution[7]*X_r[:,7]
          error_l = np.abs(Y_l - Y_hat_l)
          error_m = np.abs(Y_m - Y_hat_m)
          error_r = np.abs(Y_r - Y_hat_r)
          errors = np.sum(error_l) + np.sum(error_m) + np.sum(error_r)
          fitness = 1/(errors)
          return fitness

      return fitness_function

In [128]:
lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(m_data_l,m_data_m, m_data_r, M_l, M_m, M_r),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_genes,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        gene_type=gene_type,
                        random_seed=18)
lr_ga.run()
sol=lr_ga.best_solution()[0]
sol

array([-0.12614321,  0.78929817,  0.27933923,  1.37996731, -0.37715368,
       -0.96040415, -0.50426299,  0.5738974 ])

In [129]:
print(sol)

[-0.12614321  0.78929817  0.27933923  1.37996731 -0.37715368 -0.96040415
 -0.50426299  0.5738974 ]


In [130]:
M_hat = -0.1261 + 0.7893*X_fuz + 0.2793*W_fuz + 1.3800*Z_fuz + -0.3772*XW_fuz + -0.9604*XZ_fuz + -0.5043*WZ_fuz + 0.5739*XWZ_fuz

----

## Y 추정

In [131]:
# GA parameter 세팅

sol_per_pop = 200
num_genes = 3              # bias 포함
init_range_low = 1
init_range_high = -1


num_generations = 200
num_parents_mating = 5
gene_type = float
parent_selection_type = "rws"  # rws, rank
keep_parents = 3

crossover_type = "single_point"
mutation_type = "random"
mutation_percent_genes = 5

In [132]:
one = np.ones((1332)) # bias

In [133]:
y_data_l = np.array([one, X_l, M_l]).T
y_data_m = np.array([one, X_m, M_m]).T
y_data_r = np.array([one, X_r, M_r]).T

In [134]:
y_data_l.shape

(1332, 3)

In [135]:
def fitness_function_factory(X_l, X_m, X_r, Y_l, Y_m, Y_r):
      def fitness_function(ga_instance,solution, solution_idx):

          Y_hat_l = solution[0]*X_l[:,0] +solution[1]*X_l[:,1] +solution[2]*X_l[:,2]
          Y_hat_m = solution[0]*X_m[:,0] +solution[1]*X_m[:,1] +solution[2]*X_m[:,2]
          Y_hat_r = solution[0]*X_r[:,0] +solution[1]*X_r[:,1] +solution[2]*X_r[:,2]
          error_l = np.abs(Y_l - Y_hat_l)
          error_m = np.abs(Y_m - Y_hat_m)
          error_r = np.abs(Y_r - Y_hat_r)
          errors = np.sum(error_l) + np.sum(error_m) + np.sum(error_r)
          fitness = 1/(errors)
          return fitness

      return fitness_function

In [136]:
lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(y_data_l, y_data_m, y_data_r, Y_l, Y_m, Y_r),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_genes,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        gene_type=gene_type,
                        random_seed=18)
lr_ga.run()
sol = lr_ga.best_solution()[0]
sol

/usr/local/lib/python3.10/dist-packages/pygad/pygad.py:748: UserWarning: The percentage of genes to mutate (mutation_percent_genes=5) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


array([ 0.59886145,  0.48532108, -0.81964345])

In [137]:
print(sol)

[ 0.59886145  0.48532108 -0.81964345]


## MSE & MAE

In [138]:
one_fuz = np.ones((1332,3))

In [139]:
Y_fuz_hat = 0.5989*one_fuz + 0.4853*X_fuz + -0.8196*M_hat

* MSE

In [140]:
mse = ((Y_fuz - Y_fuz_hat) ** 2).sum(axis=1).sum() / 1332
print(mse)

0.12122206725716603


* MAE

In [141]:
abs = np.abs(Y_fuz - Y_fuz_hat)
mae = abs.sum(axis=1).sum()/len(data_18)
print(mae)

0.4916195779496951
